In [1]:
import pandas as pd
import os
import tarfile
from PIL import Image
import torchvision 
import torch.nn as nn
from torch.utils.data import TensorDataset, Dataset, DataLoader
from datetime import datetime
import torch

In [2]:
#Reading solar images
solar_Images=[f for f in os.listdir("img") if f.endswith(".jpg")]

#Loading images
transformed_Images=[]
for imageFilePath in solar_Images:
    image=Image.open("img\\"+imageFilePath)
    tensor_Transform=torchvision.transforms.ToTensor()
    transformed_Image=tensor_Transform(image)
    transformed_Images.append(transformed_Image)
    
images_Dataset=TensorDataset(transformed_Image)

In [3]:
#Reading flare dataset.
flareDataset=pd.read_csv('flareDataset.csv')
num_Rows, num_Columns=flareDataset.shape

#Setting everything to a universal time scale, with the 0 of time being the first image. 
initial_Time_Split=solar_Images[0].split("_")
date=initial_Time_Split[0]
time=initial_Time_Split[1]
s=""
s+=(f"{date[:4]}-{date[4:6]}-{date[6:]}")
s+=" "
s+=f"{time[:2]}:{time[2:4]}:{time[4:]}"
t0=datetime.strptime(s, '%Y-%m-%d %H:%M:%S')

deltat_Images=[]

for imageName in solar_Images:
    s_split=imageName.split("_")
    s="" 
    date=s_split[0]
    time=s_split[1]
    s+=(f"{date[:4]}-{date[4:6]}-{date[6:]}")
    s+=" "
    s+=f"{time[:2]}:{time[2:4]}:{time[4:]}"
    t=datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
    deltat_Images.append((t-t0).total_seconds()/3600)
    
deltat_Flares=[]
for i in range(num_Rows):
    s_split=flareDataset.loc[i, "T_REC"].split(" ")
    s="" 
    date=s_split[0]
    time=s_split[1]
    s+=date
    s+=" "
    s+=time
    t=datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
    flareDataset.loc[i, "T_REC"]=(t-t0).total_seconds()/3600
    
for i in range(num_Rows):
    if int(flareDataset.loc[i,"T_REC"])<0 or int(flareDataset.loc[i, "FlareNumber"]) !=1:
        flareDataset.drop(i, inplace=True)
   
flareDataset.reset_index(drop=True, inplace=True)

num_Rows, num_Columns=flareDataset.shape

#Normalizing flare database for gradient descent, and dropping any values that will not be useful.
flareDataset.drop("FlareNumber", axis=1, inplace=True)
flareDataset.drop("NOAA_AR", axis=1, inplace=True)
flareDataset.drop("QUALITY", axis=1, inplace=True)
flareDataset.drop("MEANPOT", axis=1, inplace=True)

means=[]
stds=[]

for col in flareDataset:
    if col!="T_REC":
        mean=flareDataset[col].mean()
        std=flareDataset[col].std()
        means.append(mean)
        stds.append(std)
        for i in range(num_Rows):
            flareDataset.loc[i, col]-=mean
            flareDataset.loc[i, col]/=std
        

C:\Users\Shiyan Liu\AppData\Local\Temp\ipykernel_3068\83276098.py:64: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '12.573684210526316' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  flareDataset.loc[i, col]-=mean
C:\Users\Shiyan Liu\AppData\Local\Temp\ipykernel_3068\83276098.py:64: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-2.1657894736842103' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  flareDataset.loc[i, col]-=mean


In [4]:

def attachActualValues():
    pointer1 = 0
    pointer2 = 0
    solarFlareDates = flareDataset.T_REC.array
    newData = []



    for tensor in transformed_Images:
        if(deltat_Images[pointer1] > solarFlareDates[pointer2]):
            pointer2 += 1

        dateOfImage = deltat_Images[pointer1]
        dateOfNextSolarFlare = solarFlareDates[pointer2]
        dateToNextFlare = dateOfNextSolarFlare-dateOfImage

        newData.append(dateToNextFlare)
        pointer1 += 1

    return newData

predictedValues=attachActualValues()

arr=[(tensor, label) for tensor, label in zip(transformed_Images, predictedValues)]

print(arr)


[(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]]), 1574.536111111111), (tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0.

In [126]:
import torchvision.transforms as transforms

#Defining custom dataset class.
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        tensor, label = self.data[index]
        if self.transform:
            tensor=self.transform(tensor)
        return tensor, label
    
#Residual neural network for regression.
resnet=torchvision.models.resnet18(pretrained="True")
resnet.float()

for param in resnet.parameters():
    param=param.to(dtype=torch.float32)
    
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

inputs=CustomDataset(arr, transform=transform_train)

num_In=resnet.fc.in_features
#Replacing classification layer with regression layer.
resnet.fc=nn.Sequential(
    nn.Linear(num_In, 256),  
    nn.ReLU(inplace=True), 
    nn.Linear(256, 256),
    nn.ReLU(inplace=True),
    nn.Linear(256, 1),
    #nn.Softmax(dim=1)
    #nn.Dropout(0.25),  
    #nn.ReLU(inplace=True), 
    #nn.Dropout(0.25),  
    #nn.Linear(256, 10),
    #nn.ReLU(inplace=True), 
    #nn.Dropout(0.25),  
    #nn.Linear(128, 10)
)
    
dataLoader=DataLoader(inputs, batch_size=16, shuffle=False)

num_Epochs=200
n=0

#Initializing model.
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
loss_Function= nn.MSELoss()

for n in range(num_Epochs):
    for batch in dataLoader:
        if len(batch[0])==dataLoader.batch_size:
            image, label=batch
            
            outputs = resnet(image) # Forward pass
            print(outputs)
            loss = loss_Function(outputs, label.to(dtype=torch.float32))  
                    
            optimizer.zero_grad()
            print((loss.item())**(1/2)/24) #Average days
            
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights
    scheduler.step()    


C:\Users\Shiyan Liu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Shiyan Liu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


tensor([[ 0.0196],
        [ 0.0148],
        [-0.1064],
        [ 0.0247],
        [-0.0182],
        [-0.1517],
        [ 0.0581],
        [ 0.0247],
        [ 0.0534],
        [-0.0097],
        [ 0.0247],
        [ 0.0247],
        [ 0.0711],
        [ 0.0632],
        [ 0.0372],
        [ 0.1176]], grad_fn=<AddmmBackward0>)
43.826626843576626
tensor([[ 0.1839],
        [ 0.0045],
        [ 0.1391],
        [ 0.0515],
        [ 0.1078],
        [ 0.1433],
        [ 0.0237],
        [ 0.1285],
        [ 0.1079],
        [-0.0686],
        [ 0.1261],
        [ 0.0726],
        [-0.0275],
        [ 0.1730],
        [ 0.1071],
        [ 0.1638]], grad_fn=<AddmmBackward0>)
58.68948567815761


C:\Users\Shiyan Liu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor([[ 0.1756],
        [-0.0652],
        [ 0.1274],
        [ 0.1643],
        [ 0.0555],
        [ 0.1312],
        [ 0.2429],
        [ 0.4453],
        [ 0.2523],
        [ 0.0493],
        [ 0.2793],
        [ 0.1926],
        [ 0.2124],
        [ 0.0519],
        [ 0.3248],
        [ 0.2562]], grad_fn=<AddmmBackward0>)
44.583515868006394
tensor([[0.1998],
        [0.2134],
        [0.1437],
        [0.2102],
        [0.2201],
        [0.2336],
        [0.3569],
        [0.2134],
        [0.2951],
        [0.2439],
        [0.2044],
        [0.0396],
        [0.0440],
        [0.2134],
        [0.2020],
        [0.2145]], grad_fn=<AddmmBackward0>)
44.407392324426745
tensor([[ 0.3450],
        [ 0.1833],
        [ 0.1365],
        [ 0.3296],
        [ 0.1969],
        [ 0.1469],
        [ 0.2751],
        [ 0.3334],
        [ 0.1574],
        [ 0.2342],
        [ 0.0874],
        [-0.0415],
        [ 0.2158],
        [ 0.2481],
        [ 0.2635],
        [ 0.1128]], grad_fn=<Ad

KeyboardInterrupt: 